In [1]:
import jax
import jax.numpy as jnp
import jax.scipy.stats as stats
import numpy as np
import sys
import blackjax

observed = np.random.normal(10, 20, size=1000)
def logdensity_fn(x):
    return jnp.sum(jnp.square(x))

# Build the kernel
step_size = 1e-3
inverse_mass_matrix = jnp.array([1., 1.])
nuts = blackjax.nuts(logdensity_fn, step_size, inverse_mass_matrix)

# Initialize the state
initial_position = jnp.array([1.0,1.0])
# {"loc": 1., "scale": 2.}
state = nuts.init(initial_position)

# Iterate
rng_key = jax.random.PRNGKey(0)
for _ in range(5):

    rng_key, nuts_key = jax.random.split(rng_key)
    state, _ = nuts.step(nuts_key, state)

I0000 00:00:1699217679.346915       1 tfrt_cpu_pjrt_client.cc:349] TfrtCpuClient created.


In [2]:
from blackjax.mcmc.mclmc import Parameters

mclmc = blackjax.mcmc.mclmc.mclmc(logdensity_fn=logdensity_fn, d = 2, transform = lambda x: x, init_key = jax.random.PRNGKey(0), params = Parameters(1,1e-3, jnp.ones(2)))



FOO


In [3]:
from blackjax.base import SamplingAlgorithm
import jax
import jax.numpy as jnp
import jax.scipy.stats as stats
import numpy as np
import sys
import blackjax
# from blackjax.types import ArrayTree

def logdensity_fn(x):
    return jnp.sum(jnp.square(x))

# Build the kernel
step_size = 1e-3
inverse_mass_matrix = jnp.array([1., 1.])
nuts = blackjax.nuts(logdensity_fn, step_size, inverse_mass_matrix)

flip = lambda f: lambda s, k : f(k,s)

def run_sampling_algorithm(sampling_algorithm : SamplingAlgorithm, num_steps : int, initial_val, rng_key):
    state = sampling_algorithm.init(initial_val)
    keys = jax.random.split(rng_key, num_steps)
    _, info = jax.lax.scan(flip(sampling_algorithm.step), state, keys)
    return info


In [4]:
run_sampling_algorithm(
    sampling_algorithm = mclmc, 
    num_steps = 4, 
    initial_val = jnp.array([1.0,1.0]),
    rng_key = jax.random.PRNGKey(0)
)

TypeError: dot requires ndarray or scalar arguments, got <class 'tuple'> at position 0.

In [3]:


run_sampling_algorithm(
    sampling_algorithm = nuts, 
    num_steps = 4, 
    initial_val = jnp.array([1.0,1.0]),
    rng_key = jax.random.PRNGKey(0)
)

NUTSInfo(momentum=Array([[ 0.09027783,  1.1181064 ],
       [-1.7013309 , -0.5390966 ],
       [ 0.770002  ,  0.6343672 ],
       [-2.1758103 , -1.742976  ]], dtype=float32), is_divergent=Array([False, False, False, False], dtype=bool), is_turning=Array([False, False,  True, False], dtype=bool), energy=Array([-1.370843  , -0.11327934, -5.84114   , -3.32897   ], dtype=float32), trajectory_leftmost_state=IntegratorState(position=Array([[1.4206688 , 0.61661047],
       [3.6223001 , 2.0243049 ],
       [2.1698742 , 1.2298268 ],
       [7.5688944 , 5.142932  ]], dtype=float32), momentum=Array([[ -1.4299476 ,   0.10284508],
       [ -5.2006354 ,  -2.6766868 ],
       [ -0.82091624,  -0.29235116],
       [-10.429538  ,  -7.2140446 ]], dtype=float32), logdensity=Array([ 2.3985085, 17.218868 ,  6.220828 , 83.73791  ], dtype=float32), logdensity_grad=Array([[ 2.8413377,  1.2332209],
       [ 7.2446003,  4.0486097],
       [ 4.3397484,  2.4596536],
       [15.137789 , 10.285864 ]], dtype=float32)